In [2]:
!pip install huggingface-hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.5/417.5 kB 7.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 38.3 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip


In [1]:
from huggingface_hub import notebook_login
notebook_login()

In [4]:
!pwd

/workspace


In [9]:
!pip install -U oyaml transformers einops albumentations python-dotenv flash_attn timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 16.7 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip


In [23]:
import requests
import torch
from PIL import Image
from transformers import AutoProcessor, AutoModelForCausalLM 
import os

device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = 'microsoft/Florence-2-large'
model = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True, torch_dtype='auto').eval().cuda()
processor = AutoProcessor.from_pretrained(model_id, trust_remote_code=True)

prompt = "<MORE_DETAILED_CAPTION>"

dataset_path = '/workspace/ai-toolkit/dataset'
valid_extensions = ('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif')
for i in os.listdir(dataset_path):
    if i.split('.')[-1] == 'txt' or not i.lower().endswith(valid_extensions):
        continue
    image = Image.open(os.path.join(dataset_path, i))

    inputs = processor(text=prompt, images=image, return_tensors="pt").to(device, torch_dtype)

    generated_ids = model.generate(
      input_ids=inputs["input_ids"],
      pixel_values=inputs["pixel_values"],
      max_new_tokens=1024,
      num_beams=3,
      do_sample=False
    )
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]

    parsed_answer = processor.post_process_generation(generated_text, task="<MORE_DETAILED_CAPTION>", image_size=(image.width, image.height))
    print(parsed_answer)
    with open(os.path.join(dataset_path, f"{i.split('.')[0]}.txt"), "w") as f:
        f.write(parsed_answer["<MORE_DETAILED_CAPTION>"])
        f.close()

{'<MORE_DETAILED_CAPTION>': 'The image is a professional headshot of a man. He is wearing a blue suit with a white shirt and a blue tie. He has a beard and short, dark hair. The man is looking directly at the camera with a slight smile on his face. The background is black, making the man the focal point of the image. He appears to be in his late 30s or early 40s.'}
{'<MORE_DETAILED_CAPTION>': 'The image is a professional headshot of a man. He is smiling and looking directly at the camera. He has a short beard and is wearing a blue suit with a white shirt and a blue tie. The background is a dark grey color. The man appears to be in his late 30s or early 40s and has a slight smile on his face.'}
{'<MORE_DETAILED_CAPTION>': 'The image is a selfie of a man sitting in a room. He is wearing a gray t-shirt with a logo on it and has a beard. He has short, dark hair and is looking directly at the camera with a slight smile on his face. In the background, there is a black cabinet with the word "

#Run Inference

In [2]:
!pip install nvitop accelerate sentencepiece protobuf peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 3.4 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip


In [6]:
!pip install -U diffusers


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip


In [8]:
!pip install -U transformers


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
!rm -rf /workspace/ai-toolkit/output

In [2]:
from nvitop import Device

devices = Device.all()  # or Device.cuda.all()
for device in devices:
    processes = device.processes()  # type: Dict[int, GpuProcess]
    sorted_pids = sorted(processes)

    print(device)
    print(f'  - Fan speed:       {device.fan_speed()}%')
    print(f'  - Temperature:     {device.temperature()}C')
    print(f'  - GPU utilization: {device.gpu_utilization()}%')
    print(f'  - Total memory:    {device.memory_total_human()}')
    print(f'  - Used memory:     {device.memory_used_human()}')
    print(f'  - Free memory:     {device.memory_free_human()}')
    print(f'  - Processes ({len(processes)}): {sorted_pids}')
    for pid in sorted_pids:
        print(f'    - {processes[pid]}')
    print('-' * 120)

PhysicalDevice(index=0, name="NVIDIA GeForce RTX 4090", total_memory=23.99GiB)
  - Fan speed:       0%
  - Temperature:     27C
  - GPU utilization: 0%
  - Total memory:    23.99GiB
  - Used memory:     1.06MiB
  - Free memory:     23.64GiB
  - Processes (0): []
------------------------------------------------------------------------------------------------------------------------


In [1]:
import torch
import gc

def flush():
    torch.cuda.empty_cache()
    gc.collect()

In [4]:
import torch
from diffusers import DiffusionPipeline

model_id = 'black-forest-labs/FLUX.1-dev'
lora_id = 'Jlonge4/jl-flux'

pipeline = DiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
weight_name = 'jl_flux_lora_v1.safetensors'
pipeline.load_lora_weights(lora_id, weight_name=weight_name)

prompt = "ethnographic photography of Josh at a picnic"
negative_prompt = "blurry, cropped, ugly"

pipeline.to('cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu')

image = pipeline(
    prompt=prompt,
    negative_prompt=negative_prompt,
    num_inference_steps=50,
    generator=torch.Generator(device='cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu').manual_seed(1641421826),
    width=1152,
    height=768,
).images[0]

display(image)

Keyword arguments {'load_in_8bit': True} are not expected by FluxPipeline and will be ignored.


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 252.00 MiB. GPU 0 has a total capacity of 23.64 GiB of which 14.81 MiB is free. Process 941856 has 23.62 GiB memory in use. Of the allocated memory 23.23 GiB is allocated by PyTorch, and 16.23 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)